In [3]:
from obspy import *
import struct
import os
import numpy as np

# 打開數據，使用 decode 指定編碼格式 ASCII 進行解碼字符串的動作
def unpackAfile(infile):
    
# == opening Afile ==
    b= os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader= struct.unpack("<4s3h6bh6s", line[0:24])
    
    fileLength = fileHeader[3]
    port = fileHeader[10]
    FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
    #print(fileHeader)
# =================================Header===================================
    
    portHeader = []
    for i in range(24,port*32,32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b",line[i:i+32])
        portHeader.append(port_data)

# =================================Data===================================    

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data=[]

    data = struct.unpack("<%di"%dataPoint,line[dataStartByte:dataStartByte+dataPoint*4])

    
    portHeader = np.array(portHeader)    
    data = np.array(data)    
    idata =data.reshape((3,port,fileLength*100),order='F')
    
#== write to obspy Stream --
    #print(fileHeader)
    #print(len(idata[0][0]))
    sttime = UTCDateTime(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    #print(sttime)
    # afst = Afile's Stream
    afst = Stream()
    
    for stc in range(fileHeader[10]):
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        #net = "TW"
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        
        # remove GPS unlock or broken station
        if ( GPS == 1 or GPS == 2 ):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            #print(chc,instrument)
            
            # for each channel in port
            for ch in range(3):
                #print(num,ch,chc)
                chn = 'Ch'+str(chc+ch)
                #print(stc,channel)
                
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp
    
    #stst = stms[0]
    #stst.detrend('linear').plot()
    #stms.write('test.mseed', format='MSEED')    
    
    return afst, FirstStn, fileHeader

In [5]:
import csv
import codecs
import time


weather = dict()
weather1 = dict()

# 創造地震資料 共20萬筆數據
with open ('network20.csv', 'w', newline = '',encoding='big5') as csvfile:
    datanames=['network','station','location','channel','npts','sampling_rate','starttime','delta','calib']
    writer = csv.DictWriter(csvfile , fieldnames = datanames)
    #writer.writeheader()
    for j in range(0,5556):
        for i in range(0,36):
            net=str(unpackAfile('05290646.40C')[0][i].stats['network'])
            stn=str(unpackAfile('05290646.40C')[0][i].stats['station'])
            loc=str(unpackAfile('05290646.40C')[0][i].stats['location'])
            chn=str(unpackAfile('05290646.40C')[0][i].stats['channel'])
            npts=str(unpackAfile('05290646.40C')[0][i].stats['npts'])
            samp=str(unpackAfile('05290646.40C')[0][i].stats['sampling_rate'])
            sttime=str(unpackAfile('05290646.40C')[0][i].stats['starttime'])
            delt=str(unpackAfile('05290646.40C')[0][i].stats['delta'])
            cal=str(unpackAfile('05290646.40C')[0][i].stats['calib'])
            weather['network']=net
            weather['station']=stn
            weather['location']=loc
            weather['channel']=chn
            weather['npts']=npts
            weather['sampling_rate']=samp
            weather['starttime']=sttime
            weather['delta']=delt
            weather['calib']=cal
            writer.writerow(weather)

In [7]:
# 創造五萬筆數據
with open("network5.csv", "w") as writer:
    reader = open("network20.csv", "r")
    for _ in range(50000):
        writer.write(reader.readline())

In [ ]:
# 計算執行所花費的時間
"""
import time
start1=time.time()

某段的hive執行程式

end=time.time()
end=end-start1
print(round(end,5))
"""

In [ ]:
from pyhive import hive  # or import hive
cursor = hive.connect(host='CT100',port=10000,auth='LDAP',username='root',password='00000', database='default').cursor()
# connect hiveserver2
# host為ubuntu主機名或ip address
# username用建立/user/hive/warehouse的授權名,要不然會出現一些授權上的問題
# database為資料庫
# password是你在執行beeline時所輸入的密碼

In [ ]:
"""
請優先照 PyHive使用分享的PPT 執行 yarn-untils.py 了解你目前需要修改的資源配置，再到mapred-site.xml 和 yarn-site.xml 進行修改 (不修改可能導致程式執行資源不足)
"""

In [ ]:
#參考資料hive指令手冊網址https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL
cursor.execute('show tables')
print (cursor.fetchall())
# 顯示 table 有哪些

In [ ]:
cursor.execute("CREATE TABLE earthquakedata1 (network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute('show tables')
print (cursor.fetchall())
# 創建 table 地震資料的格式

In [ ]:
"""
有 Partition 與 Bucket 的創建 table 方式請參考 PyHive使用分享的PPT 自行撰寫並執行
"""

In [ ]:
# partition
cursor.execute("CREATE TABLE earthquakedata2(network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) partitioned by (count INT) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute('show tables')
print (cursor.fetchall)

In [ ]:
# bucket
cursor.execute("CREATE TABLE earthquakedata3(network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) partitioned by (count INT) CLUSTERED BY(station) SORTED BY(station) INTO 11 BUCKETS row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute('show tables')
print (cursor.fetchall)

In [ ]:
cursor.execute('drop Table earthquakedata2')
cursor.execute('show tables')
print (cursor.fetchall())
# 刪除 table

In [ ]:
cursor.execute("load data local inpath '/root/network5.csv' into table earthquakedata1")
# load data inpath 是用在ubuntu主機中的資料放入在earthquake1 table 中 , 而 ''是放路徑

In [ ]:
# 這邊有兩種 insert table 的方式，請同學以自己目前要測的內容取消註解 "#" 來執行 hive 指令
import time
start = time.time()
# 一般 table 插 一般 table
#cursor.execute("insert into table earthquakedata2 select * from earthquakedata1")

# 一般 table 插 Partition table
#cursor.execute("insert into table earthquakedata2 partition (count=1) select network , station , location , channel , npts , sampling_rate , starttime , delta , calib from earthquakedata1")

# 插入 earthquakedata1 到 earthquakedata2
end=time.time()
end=end-start

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata1")
print (cursor.fetchall())
end=time.time()
end=end-start

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata2")
print (cursor.fetchall())
end=time.time()
end=end-start


In [ ]:
"""
其餘需要測試的部分請參考 PyHive 使用分享的 PPT 自行撰寫並執行
"""

In [ ]:
cursor.execute("load data local inpath '/root/network5.csv' into table earthquakedata2 PARTITION (count=1)")
# 加載5萬筆地震資料放入第二個table的 count1

In [ ]:
cursor.execute("set hive.enforce.bucketing = true")
# 命令強制執行 bucketing
import time
start = time.time()
cursor.execute("from earthquakedata2 insert overwrite table earthquakedata3 PARTITION (count=1) select network , station , location , channel , npts , sampling_rate , starttime , delta , calib WHERE count=1")
# 插入 earthquakedata2 到 earthquakedata3
end=time.time()
end=end-start

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata3")
print (cursor.fetchall())
end=time.time()
end=end-start
#query 3

In [ ]:
cursor.execute('drop Table earthquakedata1')
cursor.execute('drop Table earthquakedata2')
cursor.execute('drop Table earthquakedata3')
cursor.execute("CREATE TABLE earthquakedata1 (network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute("CREATE TABLE earthquakedata2(network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) partitioned by (count INT) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute("CREATE TABLE earthquakedata3(network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) partitioned by (count INT) CLUSTERED BY(station) SORTED BY(station) INTO 11 BUCKETS row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")

In [ ]:
cursor.execute("load data local inpath '/root/network20.csv' into table earthquakedata1")

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata1")
print (cursor.fetchall())
end=time.time()
end=end-start

In [ ]:
# 這邊有兩種 insert table 的方式，請同學以自己目前要測的內容取消註解 "#" 來執行 hive 指令
import time
start = time.time()
# 一般 table 插 一般 table
#cursor.execute("insert into table earthquakedata2 select * from earthquakedata1")

# 一般 table 插 Partition table
#cursor.execute("insert into table earthquakedata2 partition (count=1) select network , station , location , channel , npts , sampling_rate , starttime , delta , calib from earthquakedata1")

# 插入 earthquakedata1 到 earthquakedata2
end=time.time()
end=end-start

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata2")
print (cursor.fetchall())
end=time.time()
end=end-start


In [ ]:
cursor.execute("set hive.enforce.bucketing = true")
# 命令強制執行 bucketing
import time
start = time.time()
cursor.execute("from earthquakedata2 insert overwrite table earthquakedata3 PARTITION (count=1) select network , station , location , channel , npts , sampling_rate , starttime , delta , calib WHERE count=1")
# 插入 earthquakedata2 到 earthquakedata3
end=time.time()
end=end-start

In [ ]:
import time
start = time.time()
cursor.execute("select * from earthquakedata3")
print (cursor.fetchall())
end=time.time()
end=end-start
#query 3